## **Statistical Language Modeling: The "Lightweight" Autocomplete Engine**

You have been hired as a Junior NLP Engineer by a mobile software startup. The company is developing a keyboard application for low-resource feature phones that cannot run heavy Deep Learning models (like Transformers).


Your task is to build a lightweight Predictive Text Engine using statistical N-Grams. The system must learn from a corpus of text, handle words it hasn't seen frequently (Smoothing), generate coherent sentence completions, and mathematically prove its accuracy (Perplexity).


In [1]:
# Importing Required Libraries
import re
from collections import Counter, defaultdict
import random
import math
import requests
import os

#### **Task-1: N-Gram Extraction**

#### What is an N-Gram?

An N-Gram is a contiguous sequence of N items (words, characters, etc.) from a given text.

For example, in the sentence "I love machine learning", the 2-grams (bigrams) are:

["I love", "love machine", "machine learning"]

#### N-Gram Probability Equation
The probability of a word \( w_n \) given its history \( h \) in an n-gram model is:
$$
P(w_n \mid h) = \frac{C(h, w_n)}{\sum_{w'} C(h, w')}
$$
Where:
- \( C(h, w_n) \): Count of occurrences of \( h \) followed by \( w_n \)
- \( h \): Context (history of n-1 words)

<br>
<br>

##### **Task:**
1. Write a function `generate_ngrams(text, n)` that takes a corpus of text and breaks it down into contiguous sequences of `N` items.
```
Input: "I love machine learning"
Character-Level Bigrams: [('#', 'I'), ('I', ' '), (' ', 'l'), ('l', 'o'), ('o', 'v'), ('v', 'e'), ('e', ' '), (' ', 'm'), ('m', 'a'), ('a', 'c'), ('c', 'h'), ('h', 'i'), ('i', 'n'), ('n', 'e'), ('e', ' '), (' ', 'l'), ('l', 'e'), ('e', 'a'), ('a', 'r'), ('r', 'n'), ('n', 'i'), ('i', 'n'), ('n', 'g')]

```

2. Build an n-gram language model from the corpus. Write a function `build_ngram_model(corpus, n)` that takes the corpus of text and breaks it down into contiguous `N` items, and return a probability distribution for each context.

```
Input: "I love machine learning"
output: {('#',): Counter({'I': 1.0}),
             ('I',): Counter({' ': 1.0}),
             (' ',): Counter({'l': 0.6666666666666666,
                      'm': 0.3333333333333333}),
             ('l',): Counter({'o': 0.5, 'e': 0.5}),
             ('o',): Counter({'v': 1.0}),
             ('v',): Counter({'e': 1.0}),
             ('e',): Counter({' ': 0.6666666666666666,
                      'a': 0.3333333333333333}),
             ('m',): Counter({'a': 1.0}),
             ('a',): Counter({'c': 0.5, 'r': 0.5}),
             ('c',): Counter({'h': 1.0}),
             ('h',): Counter({'i': 1.0}),
             ('i',): Counter({'n': 1.0}),
             ('n',): Counter({'e': 0.3333333333333333,
                      'i': 0.3333333333333333,
                      'g': 0.3333333333333333}),
             ('r',): Counter({'n': 1.0})}
```

In [2]:
def generate_ngrams(text, n):
    p = ['#'] * (n - 1)
    ptext = p + list(text) + p
    ngrams = []
    for i in range(len(ptext) - n):
        ngrams.append(tuple(ptext[i:i+n]))
    return ngrams

In [3]:
# Example Text
text = "I love machine learning"

# Generate and display bigrams (2-grams)
bigrams = generate_ngrams(text, 2)
print("Character-Level Bigrams:", bigrams)

Character-Level Bigrams: [('#', 'I'), ('I', ' '), (' ', 'l'), ('l', 'o'), ('o', 'v'), ('v', 'e'), ('e', ' '), (' ', 'm'), ('m', 'a'), ('a', 'c'), ('c', 'h'), ('h', 'i'), ('i', 'n'), ('n', 'e'), ('e', ' '), (' ', 'l'), ('l', 'e'), ('e', 'a'), ('a', 'r'), ('r', 'n'), ('n', 'i'), ('i', 'n'), ('n', 'g')]


In [4]:
def build_ngram_model(corpus, n):
    model = defaultdict(Counter)
    ngrams = generate_ngrams(corpus, n)

    for ngram in ngrams:
        context = ngram[:-1]
        word = ngram[-1]
        model[context][word]
        model[context][word] += 1

    for context in model:
        total_count = sum(model[context].values())
        for word in model[context]:
            model[context][word] /= total_count

    return model

In [5]:
# Sample Test
corpus = "I love machine learning"
model = build_ngram_model(corpus, 2)
model

defaultdict(collections.Counter,
            {('#',): Counter({'I': 1.0}),
             ('I',): Counter({' ': 1.0}),
             (' ',): Counter({'l': 0.6666666666666666,
                      'm': 0.3333333333333333}),
             ('l',): Counter({'o': 0.5, 'e': 0.5}),
             ('o',): Counter({'v': 1.0}),
             ('v',): Counter({'e': 1.0}),
             ('e',): Counter({' ': 0.6666666666666666,
                      'a': 0.3333333333333333}),
             ('m',): Counter({'a': 1.0}),
             ('a',): Counter({'c': 0.5, 'r': 0.5}),
             ('c',): Counter({'h': 1.0}),
             ('h',): Counter({'i': 1.0}),
             ('i',): Counter({'n': 1.0}),
             ('n',): Counter({'e': 0.3333333333333333,
                      'i': 0.3333333333333333,
                      'g': 0.3333333333333333}),
             ('r',): Counter({'n': 1.0})})

#### **Task 2: Handling Sparsity (Smoothing)**
Real-world data is sparse. If a user types a word combination not found in your training data, your model returns a probability of 0, crashing the system. Implement Add-$\alpha $ (Laplace) Smoothing.

Smoothing assigns a small non-zero probability to unseen n-grams.

#### Smoothing Equation
With smoothing, the probability becomes:
$$
P(w_n \mid h) = \frac{C(h, w_n) + \alpha}{\sum_{w'} C(h, w') + \alpha \times |V|}
$$
Where:
- $\alpha $: Smoothing parameter (default is 1)
- \( |V| \): Vocabulary size


In [6]:
def add_smoothing(model, vocabulary, alpha=1.0):
    model_smoothed = defaultdict(dict)
    V = len(vocabulary)

    for context, counter in model.items():
        total_count = sum(counter.values())
        denom = total_count + alpha * V

        for word in vocabulary:
            count = counter.get(word, 0)
            model_smoothed[context][word] = (count + alpha) / denom

    return model_smoothed

In [7]:
vocabulary = set()
for counter in model.values():
    vocabulary.update(counter.keys())

model = add_smoothing(model, vocabulary, alpha=1.0)
model

defaultdict(dict,
            {('#',): {'e': 0.06666666666666667,
              'o': 0.06666666666666667,
              'v': 0.06666666666666667,
              'r': 0.06666666666666667,
              'c': 0.06666666666666667,
              'g': 0.06666666666666667,
              'l': 0.06666666666666667,
              'h': 0.06666666666666667,
              'I': 0.13333333333333333,
              ' ': 0.06666666666666667,
              'n': 0.06666666666666667,
              'm': 0.06666666666666667,
              'a': 0.06666666666666667,
              'i': 0.06666666666666667},
             ('I',): {'e': 0.06666666666666667,
              'o': 0.06666666666666667,
              'v': 0.06666666666666667,
              'r': 0.06666666666666667,
              'c': 0.06666666666666667,
              'g': 0.06666666666666667,
              'l': 0.06666666666666667,
              'h': 0.06666666666666667,
              'I': 0.06666666666666667,
              ' ': 0.13333333333333333,
     

#### **Task 3: Text Generation**
Create a function `generate_text(context, length)`:
1. Take a starting word/phrase (context).
2. Look up the probabilities of all possible next words.
3. Sample the next word based on these probabilities.
4. Update the context and repeat until the desired length is reached.



In [8]:
def generate_text(model, n, start_text, length=100):
    generated_chars = list(start_text)
    current_sequence_for_context = list(start_text)
    if len(start_text) < (n - 1):
        current_sequence_for_context = ['#'] * (n - 1 - len(start_text)) + current_sequence_for_context

    for _ in range(length - len(start_text)):
        context = tuple(current_sequence_for_context[-(n - 1):])

        next_char_probabilities = model.get(context, {})

        if not next_char_probabilities:
            all_known_chars = set()
            for k in model.keys():
                all_known_chars.update(model[k].keys())

            if all_known_chars:
                next_char = random.choice(list(all_known_chars))
            else:
                print(f"Warning: Context '{context}' not found and no known characters in model. Stopping generation.")
                break
        else:
            next_char = random.choices(list(next_char_probabilities.keys()),
                                       weights=list(next_char_probabilities.values()))[0]

        generated_chars.append(next_char)
        current_sequence_for_context.append(next_char)

    return "".join(generated_chars)

In [9]:
# Sample text
text = "hello world this is a sample text for testing the n-gram model"

# Build a bigram model
bigram_model = build_ngram_model(text, 3)

# Generate text
generated = generate_text(bigram_model, 3, "he", 15)
print(f"Generated text: {generated}")

Generated text: he n-gram model


#### **Task 4: Model Evaluation (Perplexity)**
You need to prove to your manager that your model is actually learning. Implement the Perplexity metric on a held-out test set. Lower perplexity indicates a better model.

Action: Calculate the perplexity of your model on a short unseen test sentence (e.g., "The machine learning algorithm").

#### **Perplexity**

Perplexity is a common metric for evaluating language models. Lower perplexity indicates a better model.

#### Perplexity Equation
Perplexity measures how well a language model predicts a test dataset:
$$
PP(W) = 2^{-\frac{1}{N} \sum_{i=1}^{N} \log_2 P(w_i \mid h_i)}
$$
Where:
- \( W \): Sequence of words
- \( N \): Total number of words in the sequence
- $ P(w_i \mid h_i) $: Probability of word $w_i$ given its history $ h_i $

In [10]:
def calculate_perplexity(model, n, test_text, vocabulary):
    log_sum_prob = 0.0
    N = 0
    padded_text = ['#'] * (n - 1) + list(test_text)

    for i in range(n - 1, len(padded_text)):
        context = tuple(padded_text[i - (n - 1):i])
        word = padded_text[i]
        con_prob = model.get(context, {})
        if context not in model:
            V = len(vocabulary)
            alpha = 1.0
            prob = alpha / (alpha * V)
        else:
            prob = con_prob.get(word, 0.0)
        if prob == 0:
            return float('inf')

        log_sum_prob += math.log2(prob)
        N += 1

    if N == 0:
        return float('inf')

    perplexity = 2 ** (-log_sum_prob / N)
    return perplexity

In [11]:
# First, let's create a more substantial training corpus
training_corpus = """
The quick brown fox jumps over the lazy dog.
She sells seashells by the seashore.
How much wood would a woodchuck chuck if a woodchuck could chuck wood?
To be or not to be, that is the question.
All that glitters is not gold.
A journey of a thousand miles begins with a single step.
Actions speak louder than words.
Beauty is in the eye of the beholder.
Every cloud has a silver lining.
Fortune favors the bold and brave.
Life is like a box of chocolates.
The early bird catches the worm.
Where there's smoke, there's fire.
Time heals all wounds and teaches all things.
Knowledge is power, and power corrupts.
Practice makes perfect, but nobody's perfect.
The pen is mightier than the sword.
When in Rome, do as the Romans do.
A picture is worth a thousand words.
Better late than never, but never late is better.
Experience is the best teacher of all things.
Laughter is the best medicine for the soul.
Music soothes the savage beast within us.
Nothing ventured, nothing gained in life.
The grass is always greener on the other side.
"""

# Clean the corpus
training_corpus = ''.join(c.lower() for c in training_corpus if c.isalnum() or c.isspace())

In [12]:
training_corpus

'\nthe quick brown fox jumps over the lazy dog\nshe sells seashells by the seashore\nhow much wood would a woodchuck chuck if a woodchuck could chuck wood\nto be or not to be that is the question\nall that glitters is not gold\na journey of a thousand miles begins with a single step\nactions speak louder than words\nbeauty is in the eye of the beholder\nevery cloud has a silver lining\nfortune favors the bold and brave\nlife is like a box of chocolates\nthe early bird catches the worm\nwhere theres smoke theres fire\ntime heals all wounds and teaches all things\nknowledge is power and power corrupts\npractice makes perfect but nobodys perfect\nthe pen is mightier than the sword\nwhen in rome do as the romans do\na picture is worth a thousand words\nbetter late than never but never late is better\nexperience is the best teacher of all things\nlaughter is the best medicine for the soul\nmusic soothes the savage beast within us\nnothing ventured nothing gained in life\nthe grass is always

In [13]:
def build_models(corpus):
    models = {}
    for n in [2, 3, 4]:
        raw_model = build_ngram_model(corpus, n)

        vocabulary = set()
        for context_counter in raw_model.values():
            vocabulary.update(context_counter.keys())
        vocabulary.update(list(corpus))

        smoothed_model = add_smoothing(raw_model, vocabulary, alpha=1.0)

        models[n] = {'model': smoothed_model, 'vocabulary': vocabulary}
    return models

models = build_models(training_corpus)

In [14]:
def evaluate_samples(models, num_samples=10, sample_length=40):
    results = defaultdict(list)
    for n, model_data in models.items():
        current_model = model_data['model']
        vocabulary = model_data['vocabulary']
        print(f"\nEvaluating {n}-gram model...")
        for i in range(num_samples):
            start_char = random.choice(list(vocabulary))
            generated_text = generate_text(current_model, n, start_char, length=sample_length)
            perplexity_score = calculate_perplexity(current_model, n, generated_text, vocabulary)

            results[n].append({'text': generated_text, 'perplexity': perplexity_score})
            print(f"  Sample {i+1}: '{generated_text[:20]}...' Perplexity: {perplexity_score:.2f}")
    return results

In [15]:
results = evaluate_samples(models)
print("\n=== Overall Statistics ===")
for n in models.keys():
    perplexities = [sample['perplexity'] for sample in results[n]]
    min_perp = min(perplexities)
    max_perp = max(perplexities)
    avg_perp = sum(perplexities) / len(perplexities)

    print(f"\n{n}-gram Model Statistics:")
    print(f"Minimum Perplexity: {min_perp:.2f}")
    print(f"Maximum Perplexity: {max_perp:.2f}")
    print(f"Average Perplexity: {avg_perp:.2f}")


Evaluating 2-gram model...
  Sample 1: 'vpmprxrbwsomtds 
mzs...' Perplexity: 27.96
  Sample 2: 'hopgkemrcrx
zvphxsag...' Perplexity: 28.33
  Sample 3: 'v
fkgipuzrbofvtoy
kz...' Perplexity: 28.47
  Sample 4: 'nfpdlcmjuzjm fqxapph...' Perplexity: 28.63
  Sample 5: 'fgjdsxaxnwb hyz ovkw...' Perplexity: 28.16
  Sample 6: 'zbwu kbkewvam mhaska...' Perplexity: 27.83
  Sample 7: 'bxoq otf
cstondhnczd...' Perplexity: 28.26
  Sample 8: 'vezyzbjyhctrkhspadlu...' Perplexity: 27.25
  Sample 9: 'bzmddwnxvxitblcbmbx ...' Perplexity: 28.30
  Sample 10: 'v eguxzyooximashezub...' Perplexity: 27.55

Evaluating 3-gram model...
  Sample 1: 'rxeol
sqiha
wejeqoch...' Perplexity: 29.30
  Sample 2: 'rssmitwofrljbxofswgv...' Perplexity: 28.90
  Sample 3: '#accw #i pgavilibryv...' Perplexity: 29.50
  Sample 4: 'oag d e#mvtlehqahubn...' Perplexity: 28.94
  Sample 5: 'wu
ekme#lrrgxizessgm...' Perplexity: 28.64
  Sample 6: 'wzdujhwvruymujvpy
hv...' Perplexity: 29.22
  Sample 7: 'zj#gxhs  owlznpizoer...' Perplexit

#### **Deliverables**
1. A Python Notebook (.ipynb) containing the implementations of the equations above.
2. A brief analysis report (100 words) answering: How did changing N (e.g., from Bigram to Trigram) affect the coherence of the generated text and the Perplexity score?

As N increased from a bigram (2-gram) to trigram and 4-gram models, the perplexity scores slightly increased rather than decreased. The bigram model achieved the lowest average perplexity (27.99), while trigram (28.72) and 4-gram (28.99) models showed progressively higher values. This suggests that, given the dataset size and smoothing used, higher-order models suffered from data sparsity, leading to less reliable probability estimates. In terms of coherence, bigrams produced more statistically stable but locally limited text, whereas trigrams and 4-grams captured slightly richer context but did not translate into better overall predictability, resulting in higher perplexity.